<a href="https://colab.research.google.com/github/mahbubcsedu/interviewcoding/blob/main/DP.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## <font color=green> Count the number of score combination [category: knapsack] </font>
If there are many ways to score, what would be the different combination to reach to a given score.
Lets say, 2, 3 and 7 are the scoring option, if the final score is 12, what would be the number of ways to score 12?

In [1]:
from typing import List

def num_combination_to_score(final_score: int, individual_play: List[int]):
  # initialize 2d array where the first column starts with
  # 1. Scoring 0 is possible only one way by not scoring

  dp = [[1] + [0] * final_score
        for _ in individual_play]


  for i in range(len(individual_play)):
    for j in range(1, final_score+1):
      without_this = dp[i-1][j] if i >= 1 else 0
      with_this = (dp[i][j-individual_play[i]] if j >= individual_play[i] else 0)
      combination = with_this + without_this
      dp[i][j] = combination
  return dp[len(individual_play)-1][final_score]

  print(dp)

num_combination_to_score(120, [2,3,7])

189

### <font color=green> lavenstian distance calculation </font>
source word and destination word, what is the minimum edit required to reach from source to destination
- operation is insert, delete, update
- lets represent word A or string A = A[0, n-1], and B=[0, m-1]
- we can recursively convert
  * transform A[0, n-2] to B[0, m-2] and replacing A's character by B's last character
  * transform A[0, n-2] to B[0,m-1] and then delete A's character
  * transform A[0, n-1] to B[0, m-2] and then insert B's last character at the end
  * operations are applied on A
* we can call this a equation like expression as
$ T(A[0, n-1], B[0,m-1]) = 1 + min [ T(A[0,n-1], B[0,m-1], T(A[0,n-2], B[0,m-1],T(A[0,n-1], B[0,m-2]]$

In [ ]:
def minimum_edit(source: str, destination: str) -> int :
  # print(source, destination)

  def distance_between_prefixes(A_idx, B_idx):

    # print(A_idx)
    if A_idx <0:
      return B_idx+1 # need to add all B's
    elif B_idx < 0:
      return A_idx+1 # need to delete all A's left over part


    if dp[A_idx][B_idx] == -1:
      if source[A_idx] == destination[B_idx]:
        # we have to update the dp by proceeding both 1 step
        dp[A_idx][B_idx] = distance_between_prefixes(A_idx-1, B_idx-1)
      else:
        subs = distance_between_prefixes(A_idx-1, B_idx-1)
        delete = distance_between_prefixes(A_idx-1, B_idx)
        add =  distance_between_prefixes(A_idx, B_idx-1)

        dp[A_idx][B_idx] = (1 + min(subs, add, delete))
    # print(dp)
    return dp[A_idx][B_idx]

  dp = [[-1]*len(destination) for _ in range(len(source))]
  return distance_between_prefixes(len(source)-1, len(destination)-1)

# res = minimum_edit("Carthorse", "Orchestra")
res = minimum_edit("abc", "cda")
print(res)

3


## <font color="green">Number of ways to move in a grid<font>
We can think of in reverse order. If we consider [1,1] then we can reach here from [0,1] and [1,0]. If we know the number of [0,1] and [1,0] we can calculate [1,1]
* Have to try later  
  - "Solve the same problem using O(min(n,m)) space."


In [ ]:
def num_of_ways(n, m):

  def helper(x,y):
    if x==y==0:
      return 1
    if num_ways[x][y] == 0:
      top_ways = helper(x-1, y) if x > 0 else 0
      left_ways = helper(x, y-1) if y>0 else 0
      num_ways[x][y] = top_ways + left_ways
    return num_ways[x][y]

  num_ways = [[0] * n for _ in range(m)]
  # print(num_ways)
  helper(n-1,m-1)
  print('\n'.join('     '.join(str(x) for x in row) for row in num_ways))
  return num_ways[n-1][m-1]
r = num_of_ways(5,5)
print(r)


0     1     1     1     1
1     2     3     4     5
1     3     6     10     15
1     4     10     20     35
1     5     15     35     70
70


##  <font color=green> Bionomial Coefficient </font>
$(n,k)= (n-1, k) + (n-1, k-1)$
* we can write dp in the same formula starting n, k

* it is important to know how we canculate and store. what would be the dp size. we are storing 1 based coefficient. So, we have to allocate space for 1 more than the size
* [[0]*(4) for _ in range(3)] will generate \
0  0  0  0 \
0  0  0  0 \
0  0  0  0 \
* for (n,k) we need [[0]*(k+1) for _ in range(n+1)]

In [ ]:
def bionomial_coeff(n,k):
  def helper(x,y):
    if y in (x, 0):
      return 1
    if y_out_of_x[x][y] == 0:
      without_y = helper(x-1, y-1)
      with_y = helper(x-1, y)
      y_out_of_x[x][y] = without_y + with_y
    return y_out_of_x[x][y]

  y_out_of_x=[[0]*(k+1) for _ in range(n+1)]
  print('\n'.join('  '.join(str(y) for y in row) for row in y_out_of_x))
  return helper(n,k)
  # print('\n'.join('  '.join(str(y) for y in row) for row in y_out_of_x))
print(bionomial_coeff(3,2))


0  0  0
0  0  0
0  0  0
0  0  0
3


## <font color=green> Search for a sequence of character or number in 2D arrays </font>
* Search can be starting from any point
* If we are in a point and processed prefix x which 0 to l and remaining suffix y, can we match rest of the suffix from that point?
* we will do that from each point
* this is kind of memorization and subproblem can be used to solve the actual problem. so its a dp but more like dfs

In [8]:
def is_pattern_contained_in_the_grid(grid, s):
  def dp(x,y,offset):
    if offset == len(s):
      return True

    dir = ((-1,0),(1,0),(0,1),(0,-1))
    # check if current char or number is in the current grid location
    if 0<=x<len(grid) and 0 <= y < len(grid[x]) and grid[x][y] == s[offset] and (x,y,offset) not in previous_attempt:
      if any(dp(x+a, y+b, offset+1) for a, b in dir):
        return True
    previous_attempt.add((x,y,offset))
    return False

  previous_attempt = set()

  # check for every position as starting point
  return any(dp(i,j,0)
   for i in range(len(grid)) for j in range(len(grid[i]))
   )

grid = [[1,2,3],[3,4,5],[5,6,7]]
# print(is_pattern_contained_in_the_grid(grid, [1,3,4,6]))
print(is_pattern_contained_in_the_grid(grid, [1,2,3,4]))
print(is_pattern_contained_in_the_grid(grid, [1,2,4,6,7]))

False
True


## <font color=green> Knapsack problem </font>


In [ ]:
from typing import collections
Item = collections.namedtuple("Item",('weight', 'value'))

def optimum_item_to_capacity(items, capacity):

  def helper(k, available_capacity):
    if k < 0:
      return 0
    if V[k][available_capacity] == -1:
      without_cur_item = helper(k-1, available_capacity)
      with_cur_item = (0 if available_capacity < items[k].weight else items[k].value + helper(k-1, available_capacity-items[k].weight))
      V[k][available_capacity] = max(without_cur_item, with_cur_item)
    return V[k][available_capacity]

  # lenght of first row is the number of column, we are creating row for each item and then for each item, the capacity
  V= [[-1] * (capacity+1) for _ in items]  # if
  return helper(len(items)-1, capacity)


items = [
        Item(w, v)
        for w, v in ((20, 65), (8, 35), (60, 245), (55, 195), (40, 65), (
            70, 150), (85, 275), (25, 155), (30, 120), (65, 320), (75, 75), (
                10, 40), (95, 200), (50, 100), (40, 220), (10, 99))
    ]
print(optimum_item_to_capacity(items, 130))


695


## <font color=green> Bed bath and beyond problem </font>
String constitutes of string from a dictionary (word break)

LC Wordbreak II https://leetcode.com/problems/word-break-ii/description/

A name or stirng like "bedbatchandbeyond.com" constitutes of "bed", "bath", "and" "beyond", "hand"
* build and check each prefix of the name or string, whether it is concatenation of dictionary words
* ABABC is a good example why we need dp or memorization. if we recursively check each words, we have to  check "ABC" twice, one for A, B and another for AB

Algorithm:
* its basically check all prefixes. if a prefix is part of a dictionary, we add the length at that location.
* if the prefix is not part of the dictionary, check for all suffixes until that point i. If any of the suffix is part of dictionary, update the length i-j at i location

In [ ]:
def decompose_into_dictionary_words(domain, dictionary):
  # when algorithm finishes the last_length[i] ~= -1 indicates domain[:i+1] has a valid decomposition
  # and the length of the last string in the decomposition is the last_length[i]
  # 0 based index, if domain is "abcd" domain[:1] is, domain[:2] = ab


  last_length = [-1]*len(domain)

  for i in range(len(domain)):
    # print(i, domain[:i+1])
    if domain[:i+1] in dictionary:
      last_length[i]=i+1

    if last_length[i] == -1:
      for j in range(i):
        # print("suffix", domain[j+1:i+1])
        # check this domain[j+1:i+1], its check next postion of j to ith position
        if last_length[j] != -1 and domain[j+1:i+1] in dictionary:
          last_length[i] = i-j
          break

  decompositions = []
  print(last_length)
  if last_length[-1] != -1:
    idx = len(domain)-1

    while idx >= 0:
      decompositions.append(domain[idx+1 - last_length[idx]:idx+1])
      idx -= last_length[idx]
    decompositions[::-1]

  return decompositions

dictionary = {"a","bed", "bath", "and","beyond","hand"}
res = decompose_into_dictionary_words("abedbathandbeyond", dictionary)
print(res)


[1, -1, -1, 3, -1, -1, -1, 4, 1, -1, 3, -1, -1, -1, -1, -1, 6]
['beyond', 'and', 'bath', 'bed', 'a']


In [7]:
from typing import List
from cachetools import LRUCache

def wordBreak(s: str, wordDict: List[str]) -> List[str]:
    # backtrack returns an array including all substrings derived from s.

    def backtrack(t):
      if t in mem:
        return mem[t]

      result = []
      for word in wordDict:
          if(t[0:len(word)]==word):
              next = t[len(word):]
              if(len(next)==0): result.append(word)
              else:
                  for sub in backtrack(next):
                      result.append(word+" "+sub)

      mem[s]=result
      return result
    mem={}
    return backtrack(s)

dictionary = {"a","bed", "bath", "and","beyond","hand"}
res = wordBreak("abedbathandbeyond", dictionary)
print(res)

['a bed bath and beyond']


In [5]:
from collections import Counter

def wordBreakDP(s: str, wordDict: List[str]) -> List[str]:

    #this is a quick check to see that s is not longer than all word combined

    if set(Counter(s).keys()) > set(Counter("".join(wordDict)).keys()):
        return []

    wordset = set(wordDict)

    dp=[[]]*(len(s)+1)
    dp[0] = [[0]]

    for end in range(1, len(s)+1):
        # need to keep all combinations of stops until end
        stops=[]

        for start in range(0, end):
            word = s[start:end]

            if word in wordset:
                # need to take the substops at start location and add end location to support increase until end
                for substops in dp[start]:
                    temp = substops.copy()
                    temp.append(end)
                    stops.append(temp)
        dp[end]=stops


    res = []

    # take the last position and find all stops options to create list of words

    for stops in dp[len(s)]:
        words = []

        for i in range(len(stops)-1):
            words.append(s[stops[i]:stops[i+1]])

        # add current combination
        res.append(" ".join(words))
    return res
dictionary = {"a","bed", "bath", "and","beyond","hand"}
res = wordBreakDP("abedbathandbeyond", dictionary)
print(res)

['a bed bath and beyond']


## <font color=green> Find the minimum weight path in a triagle </font>


In [ ]:
def min_path_weight(triangle):
  dp = [0]

  for row in triangle:
    print(row)
    dp = [
        row[col] +
        min(dp[max(col-1,0)], dp[min(col, len(dp)-1)])
        for col in range(len(row))
    ]
  return min(dp)
A=[[2], [4,4],[8,5,6],[4,2,6,2],[1,5,2,3,4]]

# this formula is very hard to come within few mintues of interview and need to remember, better find a way that logically comes quickly to the mind
print(min_path_weight(A))

[2]
[4, 4]
[8, 5, 6]
[4, 2, 6, 2]
[1, 5, 2, 3, 4]
15


In [ ]:
def min_path_weight(triangle):
  if len(triangle) == 0:
    return 0
  prev_row = triangle[0]

  for i in range(1, len(triangle)):
    cur_row = triangle[i]
    cur_row[0] += prev_row[0]

    for j in range(1, len(cur_row)-1):
      cur_row[j] = cur_row[j] + min(prev_row[j-1], prev_row[j])

    cur_row[len(cur_row)-1] += prev_row[len(prev_row)-1]
    prev_row = cur_row

  return min(prev_row)


A=[[2], [4,4],[8,5,6],[4,2,6,2],[1,5,2,3,4]]

# this formula is very hard to come within few mintues of interview and need to remember, better find a way that logically comes quickly to the mind
print(min_path_weight(A))

15


## <font color=green> Coin pick up game </font>
* coined are stored in a line with C[i] contains value of each coin
* player 1 and player 2 both play optimal to gain more values
* So, if we define a revenue function R(a,b) which is optimal value for the current player from coin [a, b] incluse
* as second player play optimally, we need to choose min value of second player optimum selection chance
* R(a,b) = max { {C[a] + min{ R(a+2, b), R(a+1, b-1)), (C[b] + min{R(a+1,b-1), R(a, b-2)}}

In [ ]:
def max_revenue(coins):

  def compute_max_rev_for_range(a,b):
    if a > b:
      return 0
    if max_revenue_for_range[a][b] ==0:
      max_revenue_a = coins[a] + min(compute_max_rev_for_range(a+2, b), compute_max_rev_for_range(a+1, b-1))
      max_revenue_b = coins[b] + min(compute_max_rev_for_range(a+1,b-1), compute_max_rev_for_range(a,b-2))

      max_revenue_for_range[a][b] = max(max_revenue_a, max_revenue_b)


    return max_revenue_for_range[a][b]


  max_revenue_for_range=[[0]*len(coins) for _ in range(len(coins))]
  return compute_max_rev_for_range(0, len(coins)-1)

res = max_revenue([5,25,4,1])
print(res)


26


In [ ]:
def minimum_messiness(words, line_length):
  num_remaining_blanks = line_length- len(words[0])
  # we will keep track of minimum messiness in a array, where m[i] is the minimum messiness when placing words[0:i+1]
  # so adding 0th word m[0:1], messiness is stored to m[0]
  min_messiness = ([num_remaining_blanks**2] + [float('inf')] * (len(words)-1)) # messiness to first entry and then infinite for all other

  # here we will calculate messiness after adding each words.
  # if we add a new world to the current line, and can get the current line messiness and can get first j-1 words messiness from the memory
  # after adding each words from i to 0, we check the best min messiness
  # when we finished adding last words, we will be able to get the min messiness
  for i in range(1, len(words)):
    num_remaining_blanks = line_length - len(words[i])
    min_messiness[i] = min_messiness[i-1] + num_remaining_blanks**2
    print("first loop", num_remaining_blanks, words[i])

    for j in reversed(range(i)): #for each i, continue adding words i-1, i-2, i-3 ...0 until it can affect the messiness of the current line
      num_remaining_blanks -= len(words[j])+1
      print(words[j], num_remaining_blanks)
      if num_remaining_blanks < 0:
        break
      first_j_messiness = 0 if j-1 <0 else min_messiness[j-1] # we are adding jth words, so, we can get first jith messiness from the array
      cur_line_messiness = num_remaining_blanks**2
      min_messiness[i] = min(min_messiness[i], first_j_messiness + cur_line_messiness) # if adding current words minimize messiness, update it

  print(min_messiness)
  return min_messiness[-1]

res = minimum_messiness(('aaa', 'bbb', 'c', 'd', 'ee', 'ff', 'gggggg'),11)
print(res)

first loop 8 bbb
aaa 4
first loop 10 c
bbb 6
aaa 2
first loop 10 d
c 8
bbb 4
aaa 0
first loop 9 ee
d 7
c 5
bbb 1
aaa -3
first loop 9 ff
ee 6
d 4
c 2
bbb -2
first loop 5 gggggg
ff 2
ee -1
[64, 16, 4, 0, 41, 20, 45]
45


## <font color=green> Longest increasing subsequence </font>

In [ ]:
def longest_increasing_subseq(A):
  max_length=[1]*len(A)

  for i in range(1, len(A)):
    max_length[i] = max(1 + max(
        [max_length[j] for j in range(i)
        if A[i] >=A[j]]), max_length[i])
  return max_length


def longest_increasing_subseq2(A):
  max_length=[1]*len(A)

  for i in range(1, len(A)):
    for j in range(i):
      if A[i] >=A[j]:
        max_length[i] = max(max_length[i], max_length[j]+1)
  return max_length

print(longest_increasing_subseq2([0,8,4,12,2,10,6,14,1,9]))

[1, 2, 2, 3, 2, 3, 3, 4, 2, 4]


## <font color=green> Num of ways to top </font>

In [ ]:
def num_ways_to_top(top, max_steps):
  def compute_ways_to_h(h):
    if h <=1:
      return 1

    if num_of_ways_to_h[h] == 0:
      num_of_ways_to_h[h] = sum(
          compute_ways_to_h(h-i)
          for i in range(1, min(max_steps, h)+1)
      )
    return num_of_ways_to_h[h]
  num_of_ways_to_h = [0] * (top+1)
  return compute_ways_to_h(top)
print(num_ways_to_top(4,2))

5
